# HTMX Utilities

> Utilities for handling HTMX requests and responses

In [ ]:
#| default_exp core.htmx

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Callable, Any, Optional
from fasthtml.common import *

## Request Detection

In [ ]:
#| export
def is_htmx_request(
    request # FastHTML request object
) -> bool: # True if request is from HTMX
    """Check if a request is an HTMX request."""
    return bool(request.headers.get('HX-Request'))

HTMX requests include the 'HX-Request' header set to 'true'. This is useful for conditionally returning partial HTML vs full pages.

In [ ]:
# Example: Check if request is from HTMX
from types import SimpleNamespace

# Mock HTMX request
htmx_request = SimpleNamespace(headers={'HX-Request': 'true'})
print(f"Is HTMX request: {is_htmx_request(htmx_request)}")

# Mock normal request
normal_request = SimpleNamespace(headers={})
print(f"Is HTMX request: {is_htmx_request(normal_request)}")

Is HTMX request: True
Is HTMX request: False


## HTMX Request Handler

In [ ]:
#| export
def handle_htmx_request(
    request, # FastHTML request object
    content_fn:Callable, # Function to generate content
    *args, # Positional arguments for content_fn
    wrap_fn:Optional[Callable]=None, # Optional wrapper function for full page requests
    **kwargs # Keyword arguments for content_fn
): # Content or wrapped content based on request type
    """Handle HTMX vs full page response pattern."""
    content = content_fn(*args, **kwargs)
    
    # Check if this is an HTMX request
    if is_htmx_request(request):
        return content
    
    # For full page requests, wrap with layout if provided
    if wrap_fn:
        return wrap_fn(content)
    
    return content

This is a common pattern where HTMX requests return just the content, while full page requests wrap the content with page layout.

In [ ]:
# Example: Using handle_htmx_request
from types import SimpleNamespace
from fasthtml.common import Div, H1

def my_content():
    return Div(H1("Dashboard Content"))

def my_wrapper(content):
    return Div(
        H1("Site Header"),
        content,
        Div("Footer")
    )

# HTMX request - returns just content
htmx_request = SimpleNamespace(headers={'HX-Request': 'true'})
result = handle_htmx_request(htmx_request, my_content, wrap_fn=my_wrapper)
print("HTMX request result:")
print(result)

# Full page request - returns wrapped content
normal_request = SimpleNamespace(headers={})
result = handle_htmx_request(normal_request, my_content, wrap_fn=my_wrapper)
print("\nFull page request result:")
print(result)

HTMX request result:
<div><h1>Dashboard Content</h1></div>

Full page request result:
<div><h1>Site Header</h1><div><h1>Dashboard Content</h1></div><div>Footer</div></div>


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()